## Xgboost Modelling 

In [1]:
import numpy as np
import pandas as pd

/Users/crystal/anaconda/lib/python2.7/site-packages/numexpr/cpuinfo.py:76: UserWarning: [Errno 2] No such file or directory
  stacklevel=stacklevel + 1):


In [3]:
ori_data = pd.read_csv('./lmmodel.csv',sep='\t',index_col=0)
ori_data.head()

,Month,Weekday,Hour,Temp,Rain,count,10001,10002,10003,10004,...,11428,11429,11430,11432,11433,11434,11435,11436,11439,11451
0,4,1,0,49.0,0.0,375,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,1,0,49.0,0.0,332,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,1,0,49.0,0.0,521,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,49.0,0.0,49,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,1,0,49.0,0.0,55,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
ori_data.shape

(613548, 204)

In [5]:
target = ori_data[['count']]
target.head()

,count
0,375
1,332
2,521
3,49
4,55


In [6]:
data = ori_data[[col for col in ori_data.columns if col not in ['count']]]
data.head()

,Month,Weekday,Hour,Temp,Rain,10001,10002,10003,10004,10005,...,11428,11429,11430,11432,11433,11434,11435,11436,11439,11451
0,4,1,0,49.0,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,1,0,49.0,0.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,1,0,49.0,0.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,49.0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1,0,49.0,0.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data.shape

(613548, 203)

In [9]:
import sklearn.cross_validation as cv
x_train, x_test, y_train, y_test = cv.train_test_split(data, target, 
  test_size=2.0/10, random_state=0)


In [10]:
import numpy as np
from scipy import stats
from sklearn import linear_model
ols = linear_model.LinearRegression()
ols.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [11]:
'training R^2: %.2f',ols.score(x_train, y_train)

('training R^2: %.2f', 0.74865162051584178)

In [12]:
'testing R^2: %.2f',ols.score(x_test, y_test)

('testing R^2: %.2f', 0.75619012950637643)

In [13]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, ols.predict(x_train))

16218.902688685945

In [14]:
mean_squared_error(y_test, ols.predict(x_test))

15765.279841043664

In [17]:
from __future__ import print_function
from __future__ import division
import xgboost as xgb
from sklearn.cross_validation import cross_val_score
from bayes_opt import bayesian_optimization
def xgboostcv(max_depth,
              learning_rate,
              n_estimators,
              gamma,
              min_child_weight,
              subsample,
              colsample_bytree,
              silent=True,
              nthread=-1):
    return cross_val_score(xgb.XGBRegressor(max_depth=int(max_depth),
                                            learning_rate=learning_rate,
                                            n_estimators=int(n_estimators),
                                            silent=silent,
                                            nthread=nthread,
                                            gamma=gamma,
                                            min_child_weight=min_child_weight,
                                            subsample=subsample,
                                            colsample_bytree=colsample_bytree),
                           x_train,
                           y_train,
                           "mean_squared_error",
                           cv=5).mean()

if __name__ == "__main__":
    
    xgboostBO = BayesianOptimization(xgboostcv,
                                 {'max_depth': (3, 10),
                                  'learning_rate': (0.01, 0.2),
                                  'n_estimators': (50, 500),
                                  'gamma': (1., 0.01),
                                  'min_child_weight': (1, 10),
                                  'subsample': (0.5, 1),
                                  'colsample_bytree' :(0.5, 1)})
    xgboostBO.maximize(init_points=2, n_iter = 10)
    print('-'*53)
    print('Final Results')
    print('XGBOOST: %f' % xgboostBO.res['max']['max_val'])

Initialization
--------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   learning_rate |   max_depth |   min_child_weight |   n_estimators |   subsample | 
    1 | 46m08s | -1845.22505 |             0.5381 |    0.7128 |          0.0965 |      8.6260 |             8.1723 |       437.1278 |      0.6047 | 
    2 | 26m33s | -2284.81577 |             0.7033 |    0.6865 |          0.0878 |      8.1745 |             8.0644 |       198.9211 |      0.6681 | 
Bayesian Optimization
--------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   learning_rate |   max_depth |   min_child_weight |   n_estimators |   subsample | 
    3 | 24m15s | -3375.17921 |             0.6137 |    1.0000 |          0.

In [19]:
XGB=xgb.XGBRegressor(max_depth=8,learning_rate=0.1186,n_estimators=463,silent=True,
             nthread=-1,gamma=1.0,min_child_weight=6.1929,subsample=0.9675,colsample_bytree=0.8544)

In [20]:
XGB.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8544,
       gamma=1.0, learning_rate=0.1186, max_delta_step=0, max_depth=8,
       min_child_weight=6.1929, missing=None, n_estimators=463, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.9675)

In [21]:
XGB.score(x_train, y_train)

0.9837936386988001

In [22]:
XGB.score(x_test, y_test)

0.9783336017046208

In [28]:
from sklearn.metrics import mean_squared_error

In [32]:
import math
math.sqrt(mean_squared_error(y_train, XGB.predict(x_train)))

32.33817058673971

In [33]:
math.sqrt(mean_squared_error(y_test, XGB.predict(x_test)))

37.429890915979115